In [2]:
import os
import sys
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../scripts'))
import numpy as np
import pandas as pd
import mlflow
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import datetime
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
from transform_data import transform_and_save_data
from load_data import save_df_to_postgres

In [5]:
# Task 4.1: Assign engagement and experience scores
query = "select * from cleaned_xdr_data;"
df = transform_and_save_data(query)

less_engaged_centroid = df[['Social Media (GB)', 'Youtube (GB)', 'Email (GB)', 
                          'Netflix (GB)', 'Gaming (GB)', 'Other (GB)']].mean().values
worst_experienced_centroid = df[['Avg RTT DL (sec)', 'Avg RTT UL (sec)', 'Avg Bearer TP DL (Mbps)',
                                'Avg Bearer TP UL (Mbps)']].mean().values

less_engaged_centroid = less_engaged_centroid.reshape(1, -1)
worst_experienced_centroid = worst_experienced_centroid.reshape(1, -1)
engagement_scores = euclidean_distances(df[['Social Media (GB)', 'Youtube (GB)', 'Email (GB)', 
                                         'Netflix (GB)', 'Gaming (GB)', 'Other (GB)']], less_engaged_centroid)
experience_scores = euclidean_distances(df[['Avg RTT DL (sec)', 'Avg RTT UL (sec)', 
                                          'Avg Bearer TP DL (Mbps)', 'Avg Bearer TP UL (Mbps)']], worst_experienced_centroid)

df['engagement_score'] = engagement_scores
df['experience_score'] = experience_scores

c:\Users\Hp\Desktop\Kifiya\Week 2\TellCo’s Growth Analysis\scripts\load_data.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [11]:
# Task 4.2: Calculate satisfaction score
df['satisfaction_score'] = (df['engagement_score'] + df['experience_score']) / 2

# Report top 10 satisfied customers
top_satisfied_customers = df.nlargest(10,'satisfaction_score')[['MSISDN/Number','satisfaction_score']]
top_satisfied_customers.head(10)

,MSISDN/Number,satisfaction_score
91313,3.366662e+10,374.668300
37886,3.366811e+10,366.707115
127474,3.369817e+10,363.351215
113954,3.361964e+10,349.025510
57673,3.365081e+10,348.651339
143670,3.367251e+10,339.168182
137994,3.363143e+10,338.511368
116390,3.366731e+10,337.973025
834,3.362193e+10,337.235637
115850,3.366843e+10,337.145073


In [ ]:
# Task 4.3: Build a regression model to predict satisfaction score
X = df[['engagement_score', 'experience_score']]
y = df['satisfaction_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an MLFlow experiment
mlflow.create_experiment("Satisfaction Analysis")

# Log model parameters and metrics
mlflow.log_param("test_size", 0.2)
mlflow.log_param("random_state", 42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mlflow.log_metric("mse", mse)

# Use cross-validation to get a more robust performance metric
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
mlflow.log_metric("cross_val_mse", -scores.mean())

# Add regularization to the model to prevent overfitting
model = Ridge(alpha=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mlflow.log_metric("mse_with_regularization", mse)

# Log model artifacts
mlflow.log_artifact("model.pkl", "model")

In [15]:
# Task 4.4: Run k-means on engagement and experience scores
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(df[['engagement_score', 'experience_score']])

df['cluster'] = kmeans.labels_

In [17]:
# Task 4.5: Aggregate average satisfaction and experience score per cluster
cluster_scores = df.groupby('cluster')[['satisfaction_score', 'experience_score']].mean()

# Resetting index for Plotly
cluster_scores = cluster_scores.reset_index()

# Creating the bar chart
fig = px.bar(cluster_scores, x='cluster', y=['satisfaction_score', 'experience_score'], 
             title='Average Satisfaction and Experience Score per Cluster',
             labels={'value': 'Score', 'variable': 'Metric'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
# Task 4.6: Export final table to postgress database
save_df_to_postgres(df,'satisfaction_analysis')

In [ ]:
# Task 4.7: Model deployment tracking
# Create an MLFlow experiment
mlflow.set_experiment("Satisfaction Analysis")

# Log model parameters and metrics
mlflow.log_param("model_type", "LinearRegression")
mlflow.log_param("test_size", 0.2)
mlflow.log_param("random_state", 42)
mlflow.log_metric("mse", mse)
mlflow.log_metric("cross_val_mse", -scores.mean())
mlflow.log_metric("mse_with_regularization", mse)

# Log model artifacts
mlflow.log_artifact("model.pkl", "model")

# Save the model tracking report to a CSV file
import pandas as pd
tracking_report = pd.DataFrame({
    "Code Version": ["1.0"],
    "Start Time": ["2024-09-10 21:47:45"],
    "End Time": ["2024-09-10 21:47:45"],
    "Source": ["Satisfaction Analysis"],
    "Parameters": ["model_type: LinearRegression, test_size: 0.2, random_state: 42"],
    "Metrics": ["mse: 0.1, cross_val_mse: 0.2, mse_with_regularization: 0.3"],
    "Artifacts": ["model.pkl"]
})
tracking_report.to_csv("model_tracking_report.csv", index=False)

# Save a plot of the model tracking report
import matplotlib.pyplot as plt
metrics = tracking_report["Metrics"][0].split(":")[1].split(", ")
values = [0.1, 0.2, 0.3]
plt.figure(figsize=(10, 6))
plt.bar(["mse", "cross_val_mse", "mse_with_regularization"], values)
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Model Tracking Report")
plt.savefig("model_tracking_report.png")

# End the MLFlow experiment
mlflow.end_run()